In [1]:
'''
    20251124:
        把一些神经元mask掉，再看看generation
    20251126:
        计算ppl的值


'''

'\n    20251124:\n        把一些神经元mask掉，再看看generation\n    20251126:\n        计算ppl的值\n\n\n'

In [2]:
import transformers


from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os
# download checkpoint
from accelerate import load_checkpoint_and_dispatch
from tqdm import tqdm

os.environ["TOKENIZERS_PARALLELISM"] = "false"  # To prevent long warnings :)

#from accelerate import load_checkpoint_and_dispatch

from accelerate import init_empty_weights
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer


In [3]:
import copy

In [4]:
import util
import importlib

importlib.reload(util)      # 只能 reload 模块本身
from util import calc_ppl, get_test_data   # reload 后再重新 import 函数


# function

In [5]:
def load_model(checkpoint):
    config = AutoConfig.from_pretrained(checkpoint,trust_remote_code=True)
    print('checkpoint:', checkpoint)
    
    
    if True:
    
        device_map='auto'
        model= AutoModelForCausalLM.from_pretrained(checkpoint, trust_remote_code=True, torch_dtype= torch.bfloat16,device_map=device_map ) # for download model weight
    
    else:
        with init_empty_weights():
            model = AutoModelForCausalLM.from_config(config, trust_remote_code=True)
    
        model = load_checkpoint_and_dispatch(
            model, checkpoint, device_map="auto", dtype=torch.bfloat16#, no_split_module_classes=["GPTJBlock"]
        )
    
    tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)#AutoTokenizer.from_pretrained("/home/work/lyftri/projects/model_zoo/compass_sea_13b_s4_merge2HF_org_convert_TP_1_PP_2",  trust_remote_code=True)#AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)

    return model, tokenizer

def set_neuron_zero(model, neuron_list):
    '''
    将指定的neuron设置为zero

    '''
    state_dict = model.state_dict()

    for ikey,_ in neuron_list:
        w_name = ikey.split('_index_')[0]
        neuron_index = int(ikey.split('_index_')[1])

        #set zero
        state_dict[w_name][neuron_index,:] = 0
    return model
    


def generate(model, tokenizer, textlist):
    

    
    #textlist=["The garden blooms with vibrant colors, a gentle breeze carries the scent of roses, and the sun casts a warm glow."] # maxlen 64
    #textlist=['智慧殿堂中，书籍如海，知识无尽，探索永恒。艾拉追寻历史，灵感无限']# maxlen 64
    #textlist=['你是']# maxlen 64
    
    #textlist=['The garden blooms with vibrant colors, a gentle breeze carries the scent of roses, and the sun casts a warm glow.']
    #textlist=['China is']
    bar = tqdm(total=len(textlist))
    
    
    device='cuda' # cpu cuda
    for itext in textlist:
        print('*'*20)
        input_text = itext
        print('input:', input_text)
        input_ids = tokenizer(input_text, return_tensors="pt").to(device)
        print('input_ids:', input_ids)
    
        outputs = model.generate(**input_ids, do_sample=False, num_beams=1, max_new_tokens=128) #max_length 
        print('outputs:', outputs)
        print('output:',tokenizer.batch_decode(outputs, skip_special_tokens=False))
        bar.update(1)

        
    




    

In [6]:
model, tokenizer = load_model('/root/autodl-fs/model_zoo/google/gemma-3-1b-it')

`torch_dtype` is deprecated! Use `dtype` instead!


checkpoint: /root/autodl-fs/model_zoo/google/gemma-3-1b-it


In [7]:
import torch

In [8]:
neuron_en = torch.load('/root/autodl-fs/LAPE_res/202251124_en.pt', weights_only=False)
neuron_vi = torch.load('/root/autodl-fs/LAPE_res/202251124_vi.pt', weights_only=False)
neuron_zh = torch.load('/root/autodl-fs/LAPE_res/202251124_zh.pt', weights_only=False)

In [9]:
model_en = copy.deepcopy(model)
model_en = set_neuron_zero(model_en, neuron_en)

In [10]:
model_vi = copy.deepcopy(model)
model_vi = set_neuron_zero(model_vi, neuron_vi)

In [11]:
model_zh = copy.deepcopy(model)
model_zh = set_neuron_zero(model_zh, neuron_zh)

In [12]:
# load test data
test_en = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/en_radom_1000_test.jsonl')
test_vi = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/vi_radom_1000_test.jsonl')
test_zh = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/zh_radom_1000_test.jsonl')
               
# calc ppl
model_list= [
    (model, 'org_model'),
    (model_en, 'mask_en_neuron_model'),
    (model_vi, 'mask_vi_neuron_model'),
    (model_zh, 'mask_zh_neuron_model'),
]

data_list = [
    (test_en, 'en_data'),
    (test_vi, 'vi_data'),
    (test_zh, 'zh_data')
    
]

for i_model, model_name in model_list:
    print('*'*20)
    print('*'*20)
    print('model name:', model_name)

    for i_data, data_name in data_list:
        print('='*20)
        print('data name:', data_name)
        ppl_sum, ppl_count = calc_ppl(i_model, tokenizer, i_data, max_len= 2048)
        print('ppl:',ppl_sum/ppl_count )





Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

********************
********************
model name: org_model
data name: en_data


100%|██████████| 1000/1000 [00:27<00:00, 35.79it/s]


ppl: tensor(11.2627, device='cuda:0')
data name: vi_data


...

ppl: tensor(11.4284, device='cuda:0')
data name: vi_data


100%|██████████| 1000/1000 [00:32<00:00, 30.35it/s]


ppl: tensor(11.9954, device='cuda:0')
data name: zh_data


100%|██████████| 1000/1000 [00:28<00:00, 35.35it/s]

ppl: tensor(11.4953, device='cuda:0')


## some samples

In [19]:
textlist=[
    'Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.',
    'Hãy giải thích ngắn gọn cách mô hình của bạn xử lý dữ liệu đa ngôn ngữ và làm thế nào để đảm bảo chất lượng đồng đều giữa các ngôn ngữ khác nhau.',
    '请简要说明你的模型是如何处理多语言输入的，以及你如何确保不同语言之间的输出质量保持一致。',
    'You are a climber who has reached the summit of Mount Everest. Describe your emotions and the view you see from the top.',
    'Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.',
    '你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。'
    
]

In [13]:
'''
orgmodel:
    1.Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.\n\n---\n\n**Model Description:**\n\nOur model is a transformer-based language model trained on a massive dataset of text and code in English, Spanish, French
    2.Hãy giải thích ngắn gọn cách mô hình của bạn xử lý dữ liệu đa ngôn ngữ và làm thế nào để đảm bảo chất lượng đồng đều giữa các ngôn ngữ khác nhau.\n\n**Mô hình ngôn ngữ lớn (LLM) - "LinguaVerse"**\n\nLinguaVerse là một mô hình ngôn ngữ lớn được xây
    3.请简要说明你的模型是如何处理多语言输入的，以及你如何确保不同语言之间的输出质量保持一致。\n\n**模型架构：**\n\n*   **Transformer 架构:**  我们使用基于 Transformer 的模型，特别是 BERT 和 RoBERTa 的变体。

mask en neuron:
    1.Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.\n\n}\n\n- "It" \n- "It" \n\n*   "The"\n*   "or"\n*   "It\'
    2.Hãy giải thích ngắn gọn cách mô hình của bạn xử lý dữ liệu đa ngôn ngữ và làm thế nào để đảm bảo chất lượng đồng đều giữa các ngôn ngữ khác nhau.\n\n. . .\n> . . . .\n\n. . . . . . .\n\n**\n\n**\n\n**\n\n,\n\n**\n\n,\n\n**
    3.请简要说明你的模型是如何处理多语言输入的，以及你如何确保不同语言之间的输出质量保持一致。\n\n\n\n多语言输入模型的，可以有以下这些处理方法和技术：\n\n1  1 个性化。\n2  1，1，\n

mask vi neuron:
    1.Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.\n\n**The Problem:**\n\nThe core of this is to process the input in a way that is most appropriate for the model, and to do so in a
    2.Hãy giải thích ngắn gọn cách mô hình của bạn xử lý dữ liệu đa ngôn ngữ và làm thế nào để đảm bảo chất lượng đồng đều giữa các ngôn ngữ khác nhau.\n\n---\n\nВкратце,\nПросто наберите\nЯ не\nБудьте\nВсе, что\nВы знаете\nЧто\nКак\n
    3.请简要说明你的模型是如何处理多语言输入的，以及你如何确保不同语言之间的输出质量保持一致。\n\n“请帮助 me stay silent" - It\'s a beautiful day.\nHow would you approach this problem?\nBy far, but it\'s

mask zh neuron:
    1.Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.\n\nHere re/this model explains how to handle the inconsistencies in the quality.\n\nThe following steps involve how to handle the calls to the model processes the requests
    2.Hãy giải thích ngắn gọn cách mô hình của bạn xử lý dữ liệu đa ngôn ngữ và làm thế nào để đảm bảo chất lượng đồng đều giữa các ngôn ngữ khác nhau. (إหว่างமைப்பு والأسקה האגירעות)\n(Murasala Citrix??)\n\nvề mặt máy chính ảnh posição filo? (
    3.请简要说明你的模型是如何处理多语言输入的，以及你如何确保不同语言之间的输出质量保持一致。\n\n\n\n。\n\n。。а。о。о。\n**ru。** **。**\n**。**\n**。**\n**。**

'''

'\norgmodel:\n    1.Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.\n\n---\n\n**Model Description:**\n\nOur model is a transformer-based language model trained on a massive dataset of text and code in English, Spanish, French\n    2.Hãy giải thích ngắn gọn cách mô hình của bạn xử lý dữ liệu đa ngôn ngữ và làm thế nào để đảm bảo chất lượng đồng đều giữa các ngôn ngữ khác nhau.\n\n**Mô hình ngôn ngữ lớn (LLM) - "LinguaVerse"**\n\nLinguaVerse là một mô hình ngôn ngữ lớn được xây\n    3.请简要说明你的模型是如何处理多语言输入的，以及你如何确保不同语言之间的输出质量保持一致。\n\n**模型架构：**\n\n*   **Transformer 架构:**  我们使用基于 Transformer 的模型，特别是 BERT 和 RoBERTa 的变体。\n\nmask en neuron:\n    1.Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.\n\n}\n\n- "It" \n- "It" \n\n*   "The"\n*   "or"\n*   "It\'\n    2.Hãy giải thích ngắn gọn cách mô hình của bạn xử lý dữ

In [20]:
generate(model, tokenizer, textlist)

  0%|          | 0/6 [00:00<?, ?it/s]

********************
input: Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.
input_ids: {'input_ids': tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


 17%|█▋        | 1/6 [00:02<00:14,  2.91s/it]

outputs: tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761,    108,   7243,    108,   1018,   4968,
          17243,  53121,    108,   7711,   2028,    563,    496,  29193, 236772,
           5140,   5192,   2028,  15453,    580,    496,  12566,  15297,    529,
           1816,    532,   3393,    528,   5422, 236764,  14361, 236764,   7830,
         236764,    532,   9115, 236761,   1030,  60432,    496,   5074, 236772,
         187918,  43703,   6352,    531,   2754,   1546,   5192,    618,    496,
           3550,    528,    496,   7659,   2557, 236761,    138,  14521,  34711,
         236764,    506,   2028,  52298,    506,   2148,   8369,    528,    506,
           7501, 236764,  11337,    506,   4403,    529,    506,   4251,   2744,
         236764,   2440,    506,   3738,   5192, 236761,    138,   1882,   2736,
            496,  2

 33%|███▎      | 2/6 [00:05<00:11,  2.86s/it]

...

outputs: tensor([[     2,   3048,    659,    496, 113768,   1015,    815,   8452,    506,
          34976,    529,   9348,  94509, 236761,  54234,    822,  19979,    532,
            506,   1927,    611,   1460,    699,    506,   1903, 236761,    108,
           7243,    108,    818,   6573,  54850,    657,   1041,  13925,   5716,
         236764,    496,   4512, 236764,  74693,  89830, 236761,   3551,  11762,
           3952,    528, 134880,   4314,    975, 236764,    496,   2173, 108507,
           2342,    506, 162542,  16045,    529,    506,   7217, 236761,   1701,
           5695, 236764,    564, 236858,    560,   1010,  60821,    506,   4820,
         236764,  22224,   1041,   2742,    531,   1061,  10298,   4576, 236764,
            532,   1492, 237064,   1492,    625, 236858, 236751,    784,   5367,
            625, 236761,    564, 236858,    560,   8452,    506,  34976,    529,
           9348,  94509, 236761,    108, 236776,   6794,    529,   8176, 236764,
            723,   

 83%|████████▎ | 5/6 [00:14<00:02,  2.84s/it]

outputs: tensor([[     2,  99972,   3244,   7325,  10279,  70669, 121230,  50971, 232727,
          57703, 114589,  94509, 236761,  88170,  38325, 110624,  31889,  89158,
           5042,  10044,   3884, 127379,  63719,  10044,  51414,  27888,  12835,
          14780,  21587, 236761,    108, 236769, 237703, 236748,   7439,  44693,
         100634, 236764,  22947,  31612,  13940,  31889,  89158,   3884, 127379,
          63719, 236768,    108,   1390,    108,   1390,    108,   1390,    108,
         236769, 217366,  37513,   6256,   8665,  38143,  67946,  11188,  14687,
          31889,  89158,   3884, 127379,  63719, 236768,    108,   1018, 149911,
          14435,  53121,    108, 236775, 236780,  28416,  89158,   5042,  25969,
           3244,   7325,  18457,  26220,  18234,  39652,  18457,   3375,  37814,
          30166, 236764,    566, 239731, 236743,  46524,   3884,   7325,  87062,
         132076,    534,  66928, 236761,    587,  19084,    637,  11992,  29834,
          86569,  2

100%|██████████| 6/6 [00:17<00:00,  2.86s/it]

outputs: tensor([[     2, 237408, 213062, 223874, 239972, 241888, 240344, 241454, 239963,
         239761, 239963, 236918, 151599, 237457, 236924,  47710,  19757,  22276,
         107462, 236900,  17375, 237900, 237390, 238097, 187945, 143572, 236924,
            108, 237169,  31799,  17583,  12680, 237408, 151599,  53877, 107462,
         237206, 216348, 236924,  14201, 237169,  33813,  12553, 236900,  31799,
          85694,  65407,  72245, 237622,  43719, 236924,  13488, 236900, 183868,
          21181,  22276,  19695, 236900,  92711,  71553,  31818, 236900,  85694,
         237408, 223874, 239972, 241888, 240344, 241454, 239963, 239761, 239963,
         236918,  65689, 236900, 237853,  47710, 237408, 237403, 187945, 143572,
         236924,    108, 238350,  18577, 236900,  19697,  31818, 238938, 237242,
         239852, 238450, 236900, 167467,  45370,  22276,  19695, 236924,    108,
           1018,  21480, 107462, 237184,   1018,    108, 237169,  67940,  36027,
          85629, 23

In [21]:
generate(model_en, tokenizer, textlist)

  0%|          | 0/6 [00:00<?, ?it/s]

********************
input: Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.
input_ids: {'input_ids': tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


 17%|█▋        | 1/6 [00:02<00:14,  2.94s/it]

outputs: tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761,    108, 236783,    108, 236772,    623,
           1509, 236775, 236743,    107, 236772,    623,   1509, 236775, 236743,
            108, 236829,    139, 236775,    818, 236775,    107, 236829,    139,
         236775,    504, 236775,    107, 236829,    139, 236775,   1509, 236789,
         236751, 236775,    107, 236829,    139, 236775,    509, 236789, 236751,
         236775,    107, 236829,    139, 236775,   1509, 236789, 236751, 236775,
            107, 236813,    107, 236829,    139, 236775,    509, 236789, 236751,
         236775,    107, 236829,    139, 236829,    139, 236775,   1509, 236789,
         236751, 236775,    107, 236829,    139, 236775,   1509, 236789, 236751,
         236775,    107, 236829,    139, 236775,   1509, 236789, 236751, 236775,
            107, 23

 33%|███▎      | 2/6 [00:05<00:11,  2.94s/it]

...

outputs: tensor([[     2,   3048,    659,    496, 113768,   1015,    815,   8452,    506,
          34976,    529,   9348,  94509, 236761,  54234,    822,  19979,    532,
            506,   1927,    611,   1460,    699,    506,   1903, 236761,    108,
            108,    108,    818,  34976,    522,    529,   9348,  94509,    691,
            834,  10565, 236772,    108,    108, 236772,    108, 236772,    108,
         236772,    108, 236772,    108, 236772,    108, 236772,    108, 236829,
            139, 236772,    108, 236772,    108, 236772,    108, 236829,    139,
         236829,    139, 236772,    108, 236772,    108, 236772,    108,    108,
         236772,    108,    108,    108,    108,    108,    108,    108,    108,
            108,    108,    108,    108,    108,    108,    108,    108,    108,
            108,    108,    108,    108,    108,    108,    108,    108,    108,
            108,    108,    108,    108,    108,    108,    108,    108,    108,
            108,   

 83%|████████▎ | 5/6 [00:14<00:02,  2.94s/it]

outputs: tensor([[     2,  99972,   3244,   7325,  10279,  70669, 121230,  50971, 232727,
          57703, 114589,  94509, 236761,  88170,  38325, 110624,  31889,  89158,
           5042,  10044,   3884, 127379,  63719,  10044,  51414,  27888,  12835,
          14780,  21587, 236761,    108,    108,  11388,    566,  27087,  20695,
          35619,  14780, 236786,    814,  57199,   5042,   6256,   3375,  18856,
            108, 236772,    753,    753,    808,    753,    753,    808,    108,
         236769,    829, 236754,   4611,    548,  60717, 236764,    108,    829,
         236754,   4611,    548,   5042,    108,    108, 236769, 236754,  11253,
            677, 181958,    108, 236754, 236937, 236764, 236754,   2762,   9143,
            581,    829,   4279,    108, 236754,  22214,    108, 236754, 237029,
         236757,    108,    108, 236754,    108, 236754,    108, 236749,    108,
            108, 236757,    108,    108, 236751,    108, 236763,    108, 236752,
            108, 23

100%|██████████| 6/6 [00:17<00:00,  2.94s/it]

outputs: tensor([[     2, 237408, 213062, 223874, 239972, 241888, 240344, 241454, 239963,
         239761, 239963, 236918, 151599, 237457, 236924,  47710,  19757,  22276,
         107462, 236900,  17375, 237900, 237390, 238097, 187945, 143572, 236924,
            108,    108, 236924,    108, 236924,    108,  46479, 236900,    108,
         236924,    107, 236924,    108, 236924,    108, 236924,    108, 236924,
            108,    108,    108,    108, 236924,    108, 236924, 236924,    108,
         236924,    108,    108,    108,    108, 236924,    108,    108,    108,
            108,    108,    108, 236924,    108, 236951,    108,    108,    108,
         236924,    108, 236924,    108,    108,    108,    108,    108,    108,
            108,    108,    108,    108,    108,    108,    108,    108,    108,
            108,    108,    108,    108,    108,    108,    108,    108,    108,
            108,    108,    108,    108,    108,    108,    108,    108,    108,
            108,   

In [22]:
generate(model_vi, tokenizer, textlist)

  0%|          | 0/6 [00:00<?, ?it/s]

********************
input: Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.
input_ids: {'input_ids': tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


 17%|█▋        | 1/6 [00:02<00:14,  2.94s/it]

outputs: tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761,    108,   1018,    818,  20050,  53121,
            108,    818,   7157,    529,    672,    563,    531,   1657,    506,
           2744,    528,    496,   1595,    600,    563,   1346,   6780,    573,
            506,   2028, 236764,    532,    531,    776,    834,    528,    496,
           1595,    600,    506,   2028,    740,   2480,   3050,    506,   2430,
         236789, 236751,   9703, 236761,    108,   1018,    818,  23130,  53121,
            108, 236770, 236761,   5213, 236798, 223496,  13768,  53121,    669,
           2028,   1171,   3548,    531,  18840,    506,   2430, 236789, 236751,
           9703, 236761,    107, 236778, 236761,   5213, 236798, 223496, 206043,
          53121,    669,   2028,   1299,   5724,    657,    506,  25411,    529,
            506,   

 33%|███▎      | 2/6 [00:05<00:11,  2.94s/it]

...

outputs: tensor([[     2,   3048,    659,    496, 113768,   1015,    815,   8452,    506,
          34976,    529,   9348,  94509, 236761,  54234,    822,  19979,    532,
            506,   1927,    611,   1460,    699,    506,   1903, 236761,    108,
            818,   4068,  10834,    529,    506,   5371, 236764,    532,    506,
           1202,    531,   4883,    496,   3582,    532,  16680,   4313, 236764,
            964,  64737,    531,    506,   3697,   3210, 236761,    108,   2205,
            496,   1354, 236764,    506,   2148,   2321,   4310,    529,    506,
          17085,    964,   7785,    580,    496,   4512, 236764,    532,  15334,
         236764,    107,   2574,   1411, 236747, 236786, 236828, 236814, 236832,
         236814,    107,   2094,    563,    496,  18932, 236761,    107,    108,
           1018,    818,   1346,   2132,  11441,    529,    506,    107,   2574,
           1411,   2769, 236761,  19934, 236761,    108,   1018,    818,   1346,
           2132,  1

 83%|████████▎ | 5/6 [00:14<00:02,  2.94s/it]

outputs: tensor([[     2,  99972,   3244,   7325,  10279,  70669, 121230,  50971, 232727,
          57703, 114589,  94509, 236761,  88170,  38325, 110624,  31889,  89158,
           5042,  10044,   3884, 127379,  63719,  10044,  51414,  27888,  12835,
          14780,  21587, 236761,    108,   1018,  54575,   1507, 236764,   3426,
            522,    528,    506,   4083,    529,    822,   1972,   1018,    108,
           2205,    496,   1589,   1015,    815,   7785,    506,   1774, 236743,
         236778, 236812,   3885,    529,    910,   1972, 236764, 105737,  68632,
            496,   1972,    529,    496,   3161, 236764,    532,   1299,    496,
           2321,    919,   1331, 236789, 236751, 236764,  89115, 236764,    532,
            506,  33743, 236764,    573,    506,  10298,    532,    506,   1346,
         236764,    532,   1299,    496,   2321,    919,   1331, 236789, 236751,
         236764,    532,   1299,    496,   2321,    919, 236764,    784,    506,
           1651, 23

100%|██████████| 6/6 [00:17<00:00,  2.93s/it]

outputs: tensor([[     2, 237408, 213062, 223874, 239972, 241888, 240344, 241454, 239963,
         239761, 239963, 236918, 151599, 237457, 236924,  47710,  19757,  22276,
         107462, 236900,  17375, 237900, 237390, 238097, 187945, 143572, 236924,
            108, 237227, 239026, 237982, 237105, 238267, 237026, 236900, 237227,
         239026, 236913, 237227, 236919, 237009, 239275, 238384, 237075,    107,
          14201, 237227,    107, 237009, 237921, 237410, 237437,  73242, 236918,
            107, 237227,    107, 237227,    107, 237227,    107, 237227,    107,
         237227,    107, 237227,    107, 237227,    107, 237227,    107, 237227,
            107, 237227,    107, 237227,    107, 237227,    107, 237227,    107,
         237227,    107, 237227,    107, 237227,    107, 237227,    107, 237227,
            107, 237227,    107, 237227,    107, 237227,    107, 237227,    107,
         237227,    107, 237227,    107, 237227,    107, 237227,    107, 237227,
            107, 23

In [23]:
generate(model_zh, tokenizer, textlist)

  0%|          | 0/6 [00:00<?, ?it/s]

********************
input: Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.
input_ids: {'input_ids': tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


 17%|█▋        | 1/6 [00:02<00:14,  2.92s/it]

outputs: tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761,    108,   8291,    544, 236786,   1580,
           2028,  15948,   1217,    531,   6060,    506, 114461,    528,    506,
           3325, 236761,    108,    818,   2269,   6555,  11719,   1217,    531,
           6060,    506,   9139,    531,    506,   2028,   6585,    506,  10512,
            573,    506,   1694,    529,    506,   2028,    532,    506,  13630,
            659,   4745,    531,   1607,  15579, 236761,    108,    818,   3072,
            563,    992,   2653, 236764,    840, 236764,    840,    625,    740,
            577,    496,   1401,   2455,    532,   3996,    532,    834,    580,
         236761,    108,    818,   2269,   6555,  11719,   1217,    531,   6060,
            506,   9139,    531,    506,   2028,   6585,    506,  10512,    573,
            506,   

 33%|███▎      | 2/6 [00:05<00:11,  2.92s/it]

...

outputs: tensor([[     2,   3048,    659,    496, 113768,   1015,    815,   8452,    506,
          34976,    529,   9348,  94509, 236761,  54234,    822,  19979,    532,
            506,   1927,    611,   1460,    699,    506,   1903, 236761,    108,
            898,   4977,    506,   2970,    108,   3810,   5889, 236789, 236745,
           1027,   3629,   8613,    108,   1018,   1018,   1018,   1018,    108,
           1018,   1018,   1018,    108,    108,    108,   9474,  12762,    659,
           5745,   1082,   1082,   1082,   1082,   1082,   1082,   1082,   1082,
           1082,   1082,   1082,   1082,   1082,   1082,   1082,   1082,   1082,
           1082,   1082,   1082,   5213,   1018,    108,   1018,   1018,    108,
            497,  43998,   1018,    108,   1018,   1018,   1018,    108,   1018,
            108,   1018,    108,   1018,    108,   1018,    108,   1018,   1018,
            108,    108,   1018,    108,   1018, 236772,    753,    753,    108,
           1018,   

 83%|████████▎ | 5/6 [00:14<00:02,  2.91s/it]

outputs: tensor([[     2,  99972,   3244,   7325,  10279,  70669, 121230,  50971, 232727,
          57703, 114589,  94509, 236761,  88170,  38325, 110624,  31889,  89158,
           5042,  10044,   3884, 127379,  63719,  10044,  51414,  27888,  12835,
          14780,  21587, 236761,    108,   1018,   1018,   1018,    108, 236774,
         237124, 236772,    780, 237124, 237124,  46461, 236797,   4601,    108,
         236799, 237848, 236774, 238584, 238255, 238255, 236792,    646, 238194,
         125234,  16154,   1768, 199570,   1439,  18772,   1439, 155323,   3908,
           1018, 237848, 238416, 236806,   1018, 236796,   1018,   4849, 236792,
         237642,   1740,  18309, 238194, 236814,   1018,   1018, 236773, 125395,
            603,   4169,   1439,    622, 238194,  85861,  15465, 236797,    630,
         238194,   1393,   8516,   1018, 236773,   1018,   1018,   1439,  14048,
           7171,    567,   3232,   1556,  53574,   3986,    941,   1556,  96141,
           4589,   

100%|██████████| 6/6 [00:17<00:00,  2.92s/it]

outputs: tensor([[     2, 237408, 213062, 223874, 239972, 241888, 240344, 241454, 239963,
         239761, 239963, 236918, 151599, 237457, 236924,  47710,  19757,  22276,
         107462, 236900,  17375, 237900, 237390, 238097, 187945, 143572, 236924,
            108, 242140,   7892,   6981,   5967,   5967,   5967, 101394, 113854,
           7538, 237103,  48300,  65699, 236815, 110561,    857, 218727,  23071,
           2186,  99723, 238097,   3414, 238102, 166307,   1397, 237112,    568,
         236855,   4723, 224476, 236768,  43349,  10759, 198130,  39270, 236746,
          87358,   2581,  24470,  22341,    704,  72817,    627, 242566,  25488,
          48300,    568, 236751, 236768,    138, 236769,   2057, 236808,   2680,
           2745, 236768,    138,   2057,  92248, 237287,    138, 236769,   2057,
         236808,   2680,   2745, 236768,    138,   2057, 236808,   2680,   2745,
            138,   2680,  87002,   2057,   2081,    568,   2169, 236746, 236768,
            138, 23

In [18]:
neuron_en

[('model.layers.0.mlp.up_proj.weight_index_381',
  np.float64(-9.999994999180668e-07)),
 ('model.layers.0.mlp.up_proj.weight_index_470',
  np.float64(-9.999994999180668e-07)),
 ('model.layers.0.mlp.up_proj.weight_index_2267',
  np.float64(-9.999994999180668e-07)),
 ('model.layers.0.mlp.up_proj.weight_index_4680',
  np.float64(-9.999994999180668e-07)),
 ('model.layers.2.mlp.up_proj.weight_index_342',
  np.float64(-9.999994999180668e-07)),
 ('model.layers.3.mlp.gate_proj.weight_index_2771',
  np.float64(-9.999994999180668e-07)),
 ('model.layers.3.mlp.up_proj.weight_index_1485',
  np.float64(-9.999994999180668e-07)),
 ('model.layers.3.mlp.up_proj.weight_index_2480',
  np.float64(-9.999994999180668e-07)),
 ('model.layers.3.mlp.up_proj.weight_index_3620',
  np.float64(-9.999994999180668e-07)),
 ('model.layers.3.mlp.up_proj.weight_index_6512',
  np.float64(-9.999994999180668e-07)),
 ('model.layers.4.mlp.gate_proj.weight_index_893',
  np.float64(-9.999994999180668e-07)),
 ('model.layers.4.mlp